In [2]:
from tensorflow import keras
import numpy as np
import pandas as pd
import random
import os

dataRep = '../data/'
scriptRep = '../kuzushiji_recognition/'

# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, scriptRep)
import progressBar

In [3]:
testFrac = 0.15
valFrac = 0.15

unicodeData = pd.read_csv(dataRep+'unicode_translation.csv')

raw = np.load('../data/dataset/caracterClassificationFull.npz')
image = np.expand_dims(raw['image'], 3)[:100]
charOutput = raw['characterClass'][:100]

index = np.arange(image.shape[0])
np.random.shuffle(index)
image = image[index]/255.0
charOutput = charOutput[index]

nFrac = int(image.shape[0]*(1.-testFrac))
nVal = int(image.shape[0]*(1.-testFrac-valFrac))
print(image.shape[0], image.shape[1:], nVal, nFrac)

100 (32, 32, 1) 70 85


In [6]:
def trainAndEvaluateModel(model, filenameModel, batchSize):
    
    checkpoint = keras.callbacks.ModelCheckpoint(filepath=filenameModel,
                                                 monitor='val_loss',
                                                 verbose=0,
                                                 save_best_only=True,
                                                 mode='auto', period=1)
    history = keras.callbacks.History()
    reduceLR = keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                 factor=0.1, patience=2,
                                                 verbose=0,
                                                 mode='auto')
    earlyStop = keras.callbacks.EarlyStopping(monitor='val_loss',
                                              min_delta=1e-7, patience=5,
                                              verbose=0, mode='auto')
    callbacks = [checkpoint, history, reduceLR, earlyStop]
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(image[:nVal], charOutput[:nVal], epochs=50, batch_size=batchSize,
              validation_data=(image[nVal:nFrac], charOutput[nVal:nFrac]), shuffle=True, callbacks=callbacks)
    
    model = keras.models.load_model(filenameModel)
    test_loss, test_acc = model.evaluate(image[nVal:nFrac], charOutput[nVal:nFrac])
    
    return test_loss


def optimizeHyperParameter(funcCreateModel, filenameModel, dropoutLimit=(0.,1.0), batchSizeLimit=(1,10), convolutionLayerLimit=(2,10), denseLayerLimit=(2,10), maxIter=30):
    
    i=0
    stop=False
    curentParam = [0.5, 5, 4, 8] # (dropout, batchsize, convolution, dense)
    
    stdLoss = trainAndEvaluateModel(funcCreateModel(curentParam[0], 2**(curentParam[2]), 2**(curentParam[3])), filenameModel, 2**(curentParam[1]))
    
    while (i<maxIter) and not stop:
        i+=1
        stop=True
        
        if (curentParam[0]-0.1) >= dropoutLimit[0]:
            print("Test dropout")
            testLoss = trainAndEvaluateModel(funcCreateModel(curentParam[0]-0.1, 2**(curentParam[2]), 2**(curentParam[3])),
                                             'temp.h5', 2**(curentParam[1]))
            if testLoss < stdLoss:
                stop=False
                stdLoss = testLoss
                curentParam[0] -= 0.1
                os.system("cp temp.h5 "+filenameModel)
            
        if (curentParam[0]+0.1) <= dropoutLimit[1]:
            print("Test dropout")
            testLoss = trainAndEvaluateModel(funcCreateModel(curentParam[0]+0.1, 2**(curentParam[2]), 2**(curentParam[3])),
                                             'temp.h5', 2**(curentParam[1]))
            if testLoss < stdLoss:
                stop=False
                stdLoss = testLoss
                curentParam[0] += 0.1
                os.system("cp temp.h5 "+filenameModel)
                
                
        if (curentParam[1]-1) >= batchSizeLimit[0]:
            print("Test batchsize")
            testLoss = trainAndEvaluateModel(funcCreateModel(curentParam[0], 2**(curentParam[2]), 2**(curentParam[3])),
                                             'temp.h5', 2**(curentParam[1]-1))
            if testLoss < stdLoss:
                stop=False
                stdLoss = testLoss
                curentParam[1] -= 1
                os.system("cp temp.h5 "+filenameModel)
            
        if (curentParam[1]+1) <= batchSizeLimit[1]:
            print("Test batchsize")
            testLoss = trainAndEvaluateModel(funcCreateModel(curentParam[0], 2**(curentParam[2]), 2**(curentParam[3])),
                                             'temp.h5', 2**(curentParam[1]+1))
            if testLoss < stdLoss:
                stop=False
                stdLoss = testLoss
                curentParam[1] += 1
                os.system("cp temp.h5 "+filenameModel)
                
        if (curentParam[2]-1) >= convolutionLayerLimit[0]:
            print("Test convolution")
            testLoss = trainAndEvaluateModel(funcCreateModel(curentParam[0], 2**(curentParam[2]-1), 2**(curentParam[3])),
                                             'temp.h5', 2**(curentParam[1]))
            if testLoss < stdLoss:
                stop=False
                stdLoss = testLoss
                curentParam[2] -= 1
                os.system("cp temp.h5 "+filenameModel)
            
        if (curentParam[2]+1) <= convolutionLayerLimit[1]:
            print("Test convolution")
            testLoss = trainAndEvaluateModel(funcCreateModel(curentParam[0], 2**(curentParam[2]+1), 2**(curentParam[3])),
                                             'temp.h5', 2**(curentParam[1]))
            if testLoss < stdLoss:
                stop=False
                stdLoss = testLoss
                curentParam[2] += 1
                os.system("cp temp.h5 "+filenameModel)
                
        if (curentParam[3]-1) >= denseLayerLimit[0]:
            print("Test dense")
            testLoss = trainAndEvaluateModel(funcCreateModel(curentParam[0], 2**(curentParam[2]), 2**(curentParam[3]-1)),
                                             'temp.h5', 2**(curentParam[1]))
            if testLoss < stdLoss:
                stop=False
                stdLoss = testLoss
                curentParam[3] -= 1
                os.system("cp temp.h5 "+filenameModel)
            
        if (curentParam[3]+1) <= denseLayerLimit[1]:
            print("Test dense")
            testLoss = trainAndEvaluateModel(funcCreateModel(curentParam[0], 2**(curentParam[2]), 2**(curentParam[3]+1)),
                                             'temp.h5', 2**(curentParam[1]))
            if testLoss < stdLoss:
                stop=False
                stdLoss = testLoss
                curentParam[3] += 1
                os.system("cp temp.h5 "+filenameModel)
                
    print('Dropout :', curentParam[0])
    print('Batch Size :', 2**curentParam[1])
    print('Convolutional layer :', 2**curentParam[2])
    print('Dense Layer :', 2**curentParam[3])
    os.system("rm temp.h5")
        
def createModel1(dropoutRate, convLayer, denseLayer):
    
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(convLayer, (3, 3), activation='relu', input_shape=image.shape[1:]))
    model.add(keras.layers.SpatialDropout2D(dropoutRate))
    model.add(keras.layers.Conv2D(convLayer, (3, 3), activation='relu'))
    model.add(keras.layers.SpatialDropout2D(dropoutRate))
    model.add(keras.layers.MaxPooling2D((2, 2)))
    model.add(keras.layers.Conv2D(convLayer*2, (3, 3), activation='relu'))
    model.add(keras.layers.SpatialDropout2D(dropoutRate))
    model.add(keras.layers.Conv2D(convLayer*2, (3, 3), activation='relu'))
    model.add(keras.layers.SpatialDropout2D(dropoutRate))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(denseLayer, activation='relu'))
    model.add(keras.layers.Dropout(dropoutRate))
    model.add(keras.layers.Dense(len(unicodeData), activation='softmax'))
    
    return model
    

In [7]:
optimizeHyperParameter(createModel1, '../models/KMNIST1.h5')

W0902 00:29:13.778891 140319686895424 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


Train on 70 samples, validate on 15 samples
Epoch 1/50
70/70 [==============================] - 1s 19ms/sample - loss: 8.4694 - acc: 0.0000e+00 - val_loss: 8.4737 - val_acc: 0.0000e+00
Epoch 2/50
70/70 [==============================] - 0s 3ms/sample - loss: 8.3713 - acc: 0.0429 - val_loss: 8.4531 - val_acc: 0.0000e+00
Epoch 3/50
70/70 [==============================] - 0s 3ms/sample - loss: 8.0580 - acc: 0.0857 - val_loss: 8.3301 - val_acc: 0.0000e+00
Epoch 4/50
70/70 [==============================] - 0s 2ms/sample - loss: 7.3502 - acc: 0.0714 - val_loss: 7.9830 - val_acc: 0.0000e+00
Epoch 5/50
70/70 [==============================] - 0s 2ms/sample - loss: 6.0997 - acc: 0.0714 - val_loss: 7.7092 - val_acc: 0.0000e+00
Epoch 6/50
70/70 [==============================] - 0s 2ms/sample - loss: 5.3489 - acc: 0.0857 - val_loss: 8.1884 - val_acc: 0.0000e+00
Epoch 7/50
70/70 [==============================] - 0s 3ms/sample - loss: 5.6461 - acc: 0.0286 - val_loss: 8.1343 - val_acc: 0.0000e+00

W0902 00:29:19.501131 140319686895424 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


Train on 70 samples, validate on 15 samples
Epoch 1/50
70/70 [==============================] - 2s 22ms/sample - loss: 8.4616 - acc: 0.0000e+00 - val_loss: 8.4683 - val_acc: 0.0000e+00
Epoch 2/50
70/70 [==============================] - 0s 4ms/sample - loss: 8.3626 - acc: 0.0714 - val_loss: 8.4054 - val_acc: 0.0000e+00
Epoch 3/50
70/70 [==============================] - 0s 3ms/sample - loss: 7.9192 - acc: 0.0714 - val_loss: 8.0831 - val_acc: 0.0000e+00
Epoch 4/50
70/70 [==============================] - 0s 2ms/sample - loss: 6.9220 - acc: 0.0714 - val_loss: 7.8387 - val_acc: 0.0000e+00
Epoch 5/50
70/70 [==============================] - 0s 2ms/sample - loss: 5.8737 - acc: 0.0286 - val_loss: 8.8176 - val_acc: 0.0000e+00
Epoch 6/50
70/70 [==============================] - 0s 3ms/sample - loss: 5.1920 - acc: 0.0429 - val_loss: 8.8148 - val_acc: 0.0000e+00
Epoch 7/50
70/70 [==============================] - 0s 1ms/sample - loss: 5.0785 - acc: 0.0286 - val_loss: 8.7289 - val_acc: 0.0000e+00

W0902 00:29:25.623527 140319686895424 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0902 00:29:25.663111 140319686895424 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0902 00:29:25.706104 140319686895424 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0902 00:29:25.751167 140319686895424 nn_ops.py:4224] Large dropout rate: 0.6 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


...

W0902 00:29:43.660255 140319686895424 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


Train on 70 samples, validate on 15 samples
Epoch 1/50
70/70 [==============================] - 2s 34ms/sample - loss: 8.5002 - acc: 0.0000e+00 - val_loss: 8.4478 - val_acc: 0.0000e+00
Epoch 2/50
70/70 [==============================] - 0s 2ms/sample - loss: 8.4049 - acc: 0.0000e+00 - val_loss: 8.4196 - val_acc: 0.0000e+00
Epoch 3/50
70/70 [==============================] - 0s 2ms/sample - loss: 8.2632 - acc: 0.0000e+00 - val_loss: 8.3889 - val_acc: 0.0000e+00
Epoch 4/50
70/70 [==============================] - 0s 2ms/sample - loss: 8.0723 - acc: 0.0286 - val_loss: 8.3363 - val_acc: 0.0000e+00
Epoch 5/50
70/70 [==============================] - 0s 2ms/sample - loss: 7.9159 - acc: 0.0571 - val_loss: 8.2239 - val_acc: 0.0000e+00
Epoch 6/50
70/70 [==============================] - 0s 2ms/sample - loss: 7.4462 - acc: 0.0714 - val_loss: 8.0244 - val_acc: 0.0000e+00
Epoch 7/50
70/70 [==============================] - 0s 3ms/sample - loss: 6.8784 - acc: 0.0571 - val_loss: 7.7524 - val_acc: 0.

W0902 00:29:53.821651 140319686895424 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


Train on 70 samples, validate on 15 samples


KeyboardInterrupt: 

In [5]:
dropoutRate = 0.5

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=image.shape[1:]))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(dropoutRate))
model.add(keras.layers.Dense(len(unicodeData), activation='softmax'))
model.summary()

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 30, 30, 32)        320       
_________________________________________________________________
spatial_dropout2d_4 (Spatial (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
spatial_dropout2d_5 (Spatial (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
spatial_dropout2d_6 (Spatial (None, 12, 12, 64)       

In [6]:
model.fit(image[:nFrac], charOutput[:nFrac], epochs=5, validation_split=testFrac/(1.-testFrac))
model.save('../models/KMNIST5.h5')

Train on 478424 samples, validate on 102520 samples
Epoch 1/5
478424/478424 [==============================] - 575s 1ms/sample - loss: 3.7743 - acc: 0.3494 - val_loss: 2.4794 - val_acc: 0.5603
Epoch 2/5
478424/478424 [==============================] - 565s 1ms/sample - loss: 3.0402 - acc: 0.4542 - val_loss: 2.2473 - val_acc: 0.5994
Epoch 3/5
478424/478424 [==============================] - 563s 1ms/sample - loss: 2.9663 - acc: 0.4651 - val_loss: 2.2146 - val_acc: 0.6072
Epoch 4/5
478424/478424 [==============================] - 551s 1ms/sample - loss: 2.9680 - acc: 0.4662 - val_loss: 2.1369 - val_acc: 0.6186
Epoch 5/5
478424/478424 [==============================] - 587s 1ms/sample - loss: 3.0064 - acc: 0.4611 - val_loss: 2.2947 - val_acc: 0.5976


In [7]:
dropoutRate = 0.5

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=image.shape[1:]))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.Conv2D(16, (3, 3), activation='relu'))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(dropoutRate))
model.add(keras.layers.Dense(len(unicodeData), activation='softmax'))
model.summary()

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 30, 30, 16)        160       
_________________________________________________________________
spatial_dropout2d_8 (Spatial (None, 30, 30, 16)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 28, 28, 16)        2320      
_________________________________________________________________
spatial_dropout2d_9 (Spatial (None, 28, 28, 16)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 12, 12, 32)        4640      
_________________________________________________________________
spatial_dropout2d_10 (Spatia (None, 12, 12, 32)       

In [8]:
model.fit(image[:nFrac], charOutput[:nFrac], epochs=5, validation_split=testFrac/(1.-testFrac))
model.save('../models/KMNIST2.h5')

Train on 478424 samples, validate on 102520 samples
Epoch 1/5
478424/478424 [==============================] - 337s 704us/sample - loss: 3.8546 - acc: 0.3332 - val_loss: 2.6197 - val_acc: 0.5320
Epoch 2/5
478424/478424 [==============================] - 344s 720us/sample - loss: 3.1551 - acc: 0.4318 - val_loss: 2.4001 - val_acc: 0.5679
Epoch 3/5
478424/478424 [==============================] - 357s 746us/sample - loss: 3.0688 - acc: 0.4440 - val_loss: 2.3285 - val_acc: 0.5762
Epoch 4/5
478424/478424 [==============================] - 395s 825us/sample - loss: 3.0786 - acc: 0.4433 - val_loss: 2.2172 - val_acc: 0.5955
Epoch 5/5
478424/478424 [==============================] - 342s 715us/sample - loss: 3.0537 - acc: 0.4469 - val_loss: 2.2711 - val_acc: 0.5890


In [9]:
dropoutRate = 0.5

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=image.shape[1:]))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.Conv2D(16, (3, 3), activation='relu'))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dropout(dropoutRate))
model.add(keras.layers.Dense(len(unicodeData), activation='softmax'))
model.summary()

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 30, 30, 16)        160       
_________________________________________________________________
spatial_dropout2d_12 (Spatia (None, 30, 30, 16)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 28, 28, 16)        2320      
_________________________________________________________________
spatial_dropout2d_13 (Spatia (None, 28, 28, 16)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 12, 12, 32)        4640      
_________________________________________________________________
spatial_dropout2d_14 (Spatia (None, 12, 12, 32)       

In [10]:
model.fit(image[:nFrac], charOutput[:nFrac], epochs=5, validation_split=testFrac/(1.-testFrac))
model.save('../models/KMNIST3.h5')

Train on 478424 samples, validate on 102520 samples
Epoch 1/5
478424/478424 [==============================] - 612s 1ms/sample - loss: 3.6970 - acc: 0.3668 - val_loss: 2.4662 - val_acc: 0.5723
Epoch 2/5
478424/478424 [==============================] - 630s 1ms/sample - loss: 2.9308 - acc: 0.4788 - val_loss: 2.1859 - val_acc: 0.6163
Epoch 3/5
478424/478424 [==============================] - 567s 1ms/sample - loss: 2.8272 - acc: 0.4934 - val_loss: 2.0808 - val_acc: 0.6329
Epoch 4/5
478424/478424 [==============================] - 556s 1ms/sample - loss: 2.7916 - acc: 0.4970 - val_loss: 2.0186 - val_acc: 0.6427
Epoch 5/5
478424/478424 [==============================] - 398s 832us/sample - loss: 2.7860 - acc: 0.4979 - val_loss: 2.0089 - val_acc: 0.6447


In [11]:
dropoutRate = 0.5

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=image.shape[1:]))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.Conv2D(16, (3, 3), activation='relu'))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.SpatialDropout2D(dropoutRate))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(dropoutRate))
model.add(keras.layers.Dense(len(unicodeData), activation='softmax'))
model.summary()

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 30, 30, 16)        160       
_________________________________________________________________
spatial_dropout2d_16 (Spatia (None, 30, 30, 16)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 28, 28, 16)        2320      
_________________________________________________________________
spatial_dropout2d_17 (Spatia (None, 28, 28, 16)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 12, 12, 32)        4640      
_________________________________________________________________
spatial_dropout2d_18 (Spatia (None, 12, 12, 32)       

In [12]:
model.fit(image[:nFrac], charOutput[:nFrac], epochs=5, validation_split=testFrac/(1.-testFrac))
model.save('../models/KMNIST4.h5')

Train on 478424 samples, validate on 102520 samples
Epoch 1/5
478424/478424 [==============================] - 333s 696us/sample - loss: 4.3671 - acc: 0.2399 - val_loss: 3.1050 - val_acc: 0.4387
Epoch 2/5
478424/478424 [==============================] - 330s 690us/sample - loss: 3.5764 - acc: 0.3544 - val_loss: 2.8102 - val_acc: 0.4892
Epoch 3/5
478424/478424 [==============================] - 333s 695us/sample - loss: 3.4758 - acc: 0.3690 - val_loss: 2.6842 - val_acc: 0.5058
Epoch 4/5
478424/478424 [==============================] - 337s 705us/sample - loss: 3.4492 - acc: 0.3726 - val_loss: 2.6470 - val_acc: 0.5119
Epoch 5/5
478424/478424 [==============================] - 330s 690us/sample - loss: 3.4554 - acc: 0.3718 - val_loss: 2.6807 - val_acc: 0.5082
